SVM

Logistic Regression

ANNs

KNN

Decision Tree

Naive Bayes





In [1]:
from datasets import popularity_dataset
import pandas as pd
popularity_data = popularity_dataset().dropna().drop('days_article_dataset', axis=1)

display(popularity_data)

,number_words_title,number_words_content,rate_unique_words_content,rate_non_stop_words_content,rate_unique_non_stop_words_content,number_links,number_links_to_mashable,number_imgs,number_videos,average_word_length,...,min_polarity_in_positive_words,max_polarity_in_positive_words,avg_polarity_in_negative_words,min_polarity_in_negative_words,max_polarity_in_negative_words,title_sentiment_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,12.0,219,0.663594,1.0,0.815385,4,2,1,0,4.680365,...,0.100000,0.70,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,9.0,255,0.604743,1.0,0.791946,3,1,1,0,4.913725,...,0.033333,0.70,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,9.0,211,0.575130,1.0,0.663866,3,1,1,0,4.393365,...,0.100000,1.00,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,9.0,531,0.503788,1.0,0.665635,9,0,1,0,4.404896,...,0.136364,0.80,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,13.0,1072,0.415646,1.0,0.540890,19,19,20,0,4.682836,...,0.033333,1.00,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39639,11.0,346,0.529052,1.0,0.684783,9,7,1,1,4.523121,...,0.100000,0.75,-0.260000,-0.500,-0.125000,0.100000,0.000000,0.400000,0.000000,1800
39640,12.0,328,0.696296,1.0,0.885057,9,7,3,48,4.405488,...,0.136364,0.70,-0.211111,-0.400,-0.100000,0.300000,1.000000,0.200000,1.000000,1900
39641,10.0,442,0.516355,1.0,0.644128,24,1,12,1,5.076923,...,0.136364,0.50,-0.356439,-0.800,-0.166667,0.454545,0.136364,0.045455,0.136364,1900
39642,6.0,682,0.539493,1.0,0.692661,10,1,1,0,4.975073,...,0.062500,0.50,-0.205246,-0.500,-0.012500,0.000000,0.000000,0.500000,0.000000,1100


As discussed and proposed by exploration team, we should find a way to discretize the values. Here we propose a discretization based on distribution of share values. Top 5% share values build 1 class, 80%-95% top values build class 2 etc.

In [2]:
from datasets import get_cutpoints_for_percent
classes = ['Very Bad', 'Bad', 'Moderate', 'Good', 'Very Good']

sorted_shares = popularity_data.sort_values(by=' shares', ascending= True)

classes_cutpoints = [
    get_cutpoints_for_percent(sorted_shares, 0, .3, ' shares'),
    get_cutpoints_for_percent(sorted_shares, .3, .45, ' shares'),
    get_cutpoints_for_percent(sorted_shares, .45, .8, ' shares'),
    get_cutpoints_for_percent(sorted_shares, .8, .95, ' shares'),
    get_cutpoints_for_percent(sorted_shares, .95, 1, ' shares'),
]



In [3]:
from datasets import get_class_for_value
popularity_data['share_class'] = popularity_data.apply(lambda row: get_class_for_value(row[' shares'], classes_cutpoints, classes), axis=1)

In [15]:
from datasets import get_sample_data

feature_values = popularity_data.drop(' shares', axis=1).drop('share_class', axis=1)
class_values = popularity_data['share_class']
extra_column = popularity_data[' shares']
data = get_sample_data(feature_values, class_values, extra_column, size=.5)

In [17]:
# For reference in sampling data. We see that there are 1981 entries of class 4
# Check, if sampling succeeds by checking the proportion of class 4 in all entries
count_class4 = popularity_data[popularity_data['share_class'] == classes[4]].shape[0]

# Check, if the distribution worked in the split
count_class4_split = data[data['share_class'] == classes[4]].shape[0]
# We see that of 1981 initial class 4 entries now 495 (so 25%) remain
print('Class 4 absolute before split: {}'.format(count_class4))
print('Class 4 absolute after split: {}'.format(count_class4_split))
print('Class 4 proportion before split: {:.2%}'.format(count_class4 / popularity_data.shape[0]))
print('Class 4 proportion after split: {:.2%}'.format(count_class4_split / data.shape[0]))

Class 4 absolute before split: 1981
Class 4 absolute after split: 990
Class 4 proportion before split: 5.00%
Class 4 proportion after split: 5.00%


In [18]:
data['share_class'] = data.apply(lambda row: classes.index(row['share_class']), axis=1)
x = data.drop(' shares', axis=1).drop('share_class', axis=1).dropna()
y = data['share_class'].dropna()

In [13]:
from datasets import accuracy_score
from sklearn.neural_network import MLPClassifier

annClf = MLPClassifier(hidden_layer_sizes=(5, 2,2), random_state=1)
annScore = accuracy_score(annClf,x,y)
print("Accuracy ANN: %0.2f" % annScore)

annClf = MLPClassifier(hidden_layer_sizes=(50, 50), random_state=1)
annScore = accuracy_score(annClf,x,y)
print("Accuracy ANN: %0.2f" % annScore)

Accuracy ANN: 0.34
Accuracy ANN: 0.31


In [14]:
from datasets import calculate_scores

calculate_scores(x,y)

Accuracy SVM: 0.35
Accuracy Logistic Regression: 0.37
Accuracy KNN: 0.32
Accuracy Naive Bayes: 0.34
Accuracy Decision Tree: 0.31
Accuracy ANN: 0.31
